In [1]:
#fundamental tools for the game environment model
import numpy as np
import math
import random
import itertools

# for visualations
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import exploreModel
import explore_Viewer
from randomchooser import Chooser

pygame 2.5.1 (SDL 2.28.2, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import multiprocess
from launch_async_viewer import launch_viewer

In [15]:
ch = Chooser(5, [10,10,10,10,10])

In [5]:
for k in range(35):
    ch.get_random_action()

In [6]:
ch.move_pool

array([4, 2, 1])

In [7]:
ch.hist

array([0.42857143, 0.28571429, 0.28571429])

In [8]:
env = exploreModel.exploreGame()
qq = multiprocess.Queue()

In [9]:
emap = env.generate_frame()
hmap = env.get_trace()

In [10]:
qq.put((emap, hmap))

In [11]:
p = multiprocess.Process(target=launch_viewer, args=[qq])

In [12]:
p.start()

In [16]:
for k in range(1000000):
    env.update(ch.get_random_action())
    emap = env.generate_frame()
    hmap = env.get_trace()

    if qq.empty():
        qq.put((emap, hmap))
    
    if k % 150 == 149:
        env.soft_reset()

In [10]:
qq.put("kill")